In [1]:
import matplotlib.pyplot as plt
import numpy as np

from scipy.integrate import solve_ivp
from scipy.optimize import curve_fit
from ipywidgets import interact

In [11]:
# define function for derivative
def dvdt(t, v, tau):
    return -v/tau

# define exact solution
def ex_sol(t, A, tau):
    return A * np.exp(-t / tau)

# define parameters
L = 0.1
B = 0.5
R = 1
m = 0.05

v0 = 1

tau = (R*m)/(L*B)**2

tmax = 5*tau

@interact(N=(1, 100), RK=True, Fit=True, Euler=False, Exact=False)
def plot(N=10, RK=True, Fit=True, Euler=False, Exact=False):

    dt = tmax/N
    t = np.linspace(0, tmax, N+1) # array for times
    v_eu = np.zeros(N+1) # prepare empty array for positions

    N_ex = 100 # number of points to plot for exact solution
    t_ex = np.linspace(0, tmax, N_ex) # times for exact solution
    v_ex = ex_sol(t_ex, v0, tau) # positions for exact solution
    v_ex2 = ex_sol(t, v0, tau) # positions at times for numerical solution (for comparison)

    # Euler algorithm
    v_eu[0] = v0

    for i in range(0, N):
        slope = dvdt(0, v_eu[i], tau) # get value for slope of current point
        v_eu[i+1] = v_eu[i] + slope * dt # calculate position of next step

    # Runge-Kutta algorithm
    rk = solve_ivp(dvdt, (0, tmax), [v0], args=[tau], t_eval=t) # call solve_ivp to get solution with RK
    v_rk = rk.y[0] # assign positions (rk.t contains times, rk.y positions)

    # Fit to RK
    param, cov = curve_fit(ex_sol, t, v_rk)
    v0_fit, tau_fit = param

    v_fit = ex_sol(t_ex, v0_fit, tau_fit)

    # prepare stacked plot
    fig, axs = plt.subplots(2, figsize=(8, 8), gridspec_kw={'height_ratios': [2, 1]})
    ax1, ax2 = axs

    tscale = 1 # scale factor for time (µs)
    Iscale = 1 # scale factor for current (mA)
    
    if RK:
        ax1.scatter(t*tscale, v_rk*Iscale, s=4, c='black', label="Runge-Kutta")

    if Fit:
        ax1.plot(t_ex*tscale, v_fit*Iscale, 'g--', linewidth=1, label='fit to RK')
        ax1.text(3.5*tau*tscale, v0_fit/5*Iscale, f'fit parameters: \n'
                 +r'$I_0$'+f' = {v0_fit:.3f} m/s \n'
                 +r'$\tau$'+f' = {tau_fit:.3f} s')
    
    if Euler:
        ax1.scatter(t*tscale, v_eu*Iscale, s=4, c='red', label='Euler')
    
    if Exact:
        ax1.plot(t_ex*tscale, v_ex*Iscale, 'b--', linewidth=1, label='exact solution')
        ax1.text(2*tau*tscale, v0/5*Iscale, f'exact values: \n'
                 +r'$I_0$'+f' = {v0:.3f} m/s \n'
                 +r'$\tau$'+f' = {tau:.3f} s')
    
    ax1.set(xlabel='Time (s)', ylabel='Velocity (m/s)')
    ax1.grid()
    ax1.legend()

    if Euler:
        diff = v_eu-v_ex2
        ax2.scatter(t*tscale, np.divide(diff, v_ex2, out=np.zeros_like(diff), where=v_ex2!=0), 
                    s=4, c='red', label='Euler – exact')
    if RK:
        diff = v_rk-v_ex2
        ax2.scatter(t*tscale, np.divide(diff, v_ex2, out=np.zeros_like(diff), where=v_ex2!=0), 
                    s=4, c='black', label='RK – exact')
    ax2.grid()
    ax2.legend()
    plt.show()


interactive(children=(IntSlider(value=10, description='N', min=1), Checkbox(value=True, description='RK'), Che…